In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np

from catboost import CatBoostRegressor, CatBoostClassifier
from sklearn.metrics import mean_squared_error


from matplotlib import pyplot as plt
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

In [ ]:
df = pd.read_csv('/kaggle/input/real-time-advertisers-auction/Dataset.csv')

In [ ]:
df['date'] = df.date.apply(lambda l: pd.Timestamp(l).value)

In [ ]:
def weird_division(n, d):
    return n / d if d else 0

df['CPM'] = df.apply(lambda x: weird_division(((x['total_revenue'] * 100)), x['measurable_impressions']) * 1000 , axis=1)

In [ ]:
train = df[df.date < pd.Timestamp('06-21-2019').value]
test = df[df.date >= pd.Timestamp('06-21-2019').value]

Y_train = df[(df.date < pd.Timestamp('06-21-2019').value)].CPM
Y_test = df[(df.date >= pd.Timestamp('06-21-2019').value)].CPM

In [ ]:
Y_test = Y_test[test['CPM'].between(-1e-10, test['CPM'].quantile(.95))]
test = test[test['CPM'].between(-1e-10, test['CPM'].quantile(.95))]

In [ ]:
train.drop(labels=['total_revenue', 'measurable_impressions', 'order_id' , 'date'], axis=1, inplace=True)
test.drop(labels=['total_revenue', 'measurable_impressions', 'order_id' , 'date'], axis=1, inplace=True)

# Some EDA

In [ ]:
plt.figure(figsize=(15, 10))
plt.hist(train.CPM, bins=200);

In plotted histogram we can see too much zero values.

Then let's build next pipeline:

-fit classificator to detect zero CPM

-fit regressor to predict non-zero CPM

97 percentile gives more data and removes too big CPM

In [ ]:
Y_train = Y_train[train['CPM'].between(-1e-10, test['CPM'].quantile(.97))]
train = train[train['CPM'].between(-1e-10, test['CPM'].quantile(.97))]

In [ ]:
train['is_non_zero'] = train.CPM != 0

In [ ]:
train.is_non_zero.value_counts(normalize=True)

Almost balansed target for classification task!

In [ ]:
train.drop(labels=['CPM'], axis=1, inplace=True)
test.drop(labels=['CPM'], axis=1, inplace=True)

In [ ]:
train.head()

# Fit models

In [ ]:
cat_cols = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [ ]:
X_train_clf, X_val_clf, y_train_clf, y_val_clf = train_test_split(train, train.is_non_zero, test_size=0.1, random_state=42)
X_train_clf.drop(labels=['is_non_zero'], axis=1, inplace=True)
X_val_clf.drop(labels=['is_non_zero'], axis=1, inplace=True)

In [ ]:
xgb_clf = CatBoostClassifier(n_estimators=2000, learning_rate=0.03, eval_metric='Accuracy', metric_period=50)
xgb_clf.fit(X_train_clf, y_train_clf, eval_set=(X_val_clf, y_val_clf), cat_features = cat_cols, early_stopping_rounds=100)

In [ ]:
def optimize_f1(clf, X, y):
    scores = [(g, f1_score(y, clf.predict_proba(X)[:,1:] > g)) for g in np.linspace(0, 1, 20)]
    return sorted(scores, key=lambda l: -l[1])[0][0]

In [ ]:
board = optimize_f1(xgb_clf, X_val_clf, y_val_clf)

In [ ]:
X_train_reg, X_val_reg, y_train_reg, y_val_reg = train_test_split(train[train.is_non_zero], Y_train[train.is_non_zero], test_size=0.1, random_state=42)
X_train_reg.drop(labels=['is_non_zero'], axis=1, inplace=True)
X_val_reg.drop(labels=['is_non_zero'], axis=1, inplace=True)

In [ ]:
xgb = CatBoostRegressor(eval_metric='RMSE', loss_function='RMSE', metric_period=50)
xgb.fit(X_train_reg, y_train_reg, eval_set=(X_val_reg, y_val_reg), cat_features = cat_cols, early_stopping_rounds=100)

# Inference

In [ ]:
clf_preds = np.squeeze(xgb_clf.predict_proba(test)[:,1:] > board)

In [ ]:
submit = np.zeros(test.shape[0])

In [ ]:
submit[clf_preds] = xgb.predict(test[clf_preds])

In [ ]:
predicted_mse = mean_squared_error(submit, Y_test)
predicted_mse

In [ ]:
expected_mse = 4850

In [ ]:
if predicted_mse < expected_mse:
    print("OK")
else:
    print("Akshay WIN!")